##*Modelinho* Ok


In [3]:
#Bibliotecas
import os
import cv2
import numpy as np
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
drive.mount('/content/drive')

# Caminho da pasta raiz contendo as subpastas
dataset = "/content/drive/MyDrive/IA_Projects/Face_Recognition/faces_dataset/partial_dataset"

# Rotula dataset
labels = []
for foldername in os.listdir(dataset):
    labels.append(foldername)
labels = np.array(labels)
#print("Número de rótulos encontrados:", len(labels))
#print("Rótulos encontrados:", labels)
#np.save("rotulos.npy", labels)

# Define numero de pessoas no dataset
n_classes = len(labels)

In [2]:
datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2, #cisalhamento - trans. geometrica que distorce a imagem e simula angulação em imagens reais 0.2rad
                                   zoom_range=0.2,
                                   horizontal_flip=True) # tecnica de espelhamento, serve para aumentar os dados e treinar o modelo para angulos diferentes

num_samples = len(datagen.flow_from_directory(dataset, batch_size=16, shuffle=False).filenames)

# Ajuste de formato das imagens
train_generator = datagen.flow_from_directory(
    
    dataset,
    target_size=(112, 112),
    batch_size=16,
    shuffle=False,
    class_mode='categorical')

x, y = train_generator.next()

X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)


Found 2484 images belonging to 579 classes.
Found 2484 images belonging to 579 classes.


In [9]:
model = tf.keras.models.Sequential([
    # primeira camada convolucional
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # segunda camada convolucional
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # terceira camada convolucional
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # quarta camada convolucional
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(n_classes, activation='softmax')])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [11]:
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_val, y_val))


Epoch 1/20
1/1 [==============================] - 1s 730ms/step - loss: 0.7455 - categorical_accuracy: 0.7500 - val_loss: 9.0982 - val_categorical_accuracy: 0.2500
Epoch 2/20
1/1 [==============================] - 1s 990ms/step - loss: 0.7187 - categorical_accuracy: 0.6667 - val_loss: 10.4374 - val_categorical_accuracy: 0.2500
Epoch 3/20
1/1 [==============================] - 1s 1s/step - loss: 0.7052 - categorical_accuracy: 0.6667 - val_loss: 11.4915 - val_categorical_accuracy: 0.2500
Epoch 4/20
1/1 [==============================] - 1s 692ms/step - loss: 0.5957 - categorical_accuracy: 0.6667 - val_loss: 12.2225 - val_categorical_accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 384ms/step - loss: 0.7148 - categorical_accuracy: 0.5833 - val_loss: 12.5338 - val_categorical_accuracy: 0.2500
Epoch 6/20
1/1 [==============================] - 0s 406ms/step - loss: 0.4963 - categorical_accuracy: 0.8333 - val_loss: 12.8504 - val_categorical_accuracy: 0.2500
Epoch 7/20
1/1

In [8]:
from sklearn.metrics import confusion_matrix

# Faz as previsões no conjunto de teste
y_pred = np.argmax(model.predict(X_val), axis=-1)

# Cria a matriz de confusão
cm = confusion_matrix(y_val, y_pred)
print(cm)


1/1 [==============================] - 0s 177ms/step


ValueError: ignored

##Reconhecimento facial com distancia euclidiana




Este modelo é formado por duas redes neurais idênticas, que compartilham seus pesos e recebem duas imagens de entrada: uma imagem de referência e uma imagem de comparação. As imagens passam por camadas convolucionais e de pooling, e então são passadas para uma camada densa de embedding com tamanho embedding_dim. As embeddings resultantes de cada imagem são normalizadas para terem norma unitária.

Finalmente, é calculada a distância euclidiana entre as embeddings resultantes das duas imagens. Este modelo pode ser treinado utilizando um conjunto de pares de imagens, onde cada par é rotulado com uma distância (0 se as imagens são iguais, 1 se são diferentes). O objetivo do treinamento é minimizar a perda da distância entre as embeddings em pares de imagens correspondentes e maximizá-la em pares de imagens não correspondentes.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Definindo a função para calcular a distância euclidiana
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

# Definindo o modelo
def create_model(input_shape, embedding_dim):
    input_shape = (input_shape[0], input_shape[1], 3)
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Camadas convolucionais e de pooling
    x = Conv2D(32, (3, 3), activation='relu')(input_a)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(embedding_dim, activation='relu')(x)
    embeddings_a = Lambda(lambda x: K.l2_normalize(x, axis=1))(x)

    x = Conv2D(32, (3, 3), activation='relu')(input_b)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(embedding_dim, activation='relu')(x)
    embeddings_b = Lambda(lambda x: K.l2_normalize(x, axis=1))(x)

    # Calculando a distância euclidiana entre as embeddings
    distance = Lambda(euclidean_distance)([embeddings_a, embeddings_b])

    # Definindo o modelo final
    model = Model(inputs=[input_a, input_b], outputs=distance)
    return model


##Reconhecimento facial com distancia de mahalanobis


Neste exemplo, a distância de Mahalanobis é calculada dentro da função triplet_loss, que é utilizada como função de perda do modelo. Note que a função mahalanobis_dist é utilizada para calcular a distância entre as embeddings. Além disso, foi necessário importar a classe LedoitWolf do módulo sklearn.covariance para calcular a matriz de covariância empírica.





In [ ]:
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import LedoitWolf

def mahalanobis_dist(x1, x2):
    # Calcula a matriz de covariância
    cov = LedoitWolf().fit(x1).covariance_
    # Calcula a matriz de covariância inversa
    cov_inv = np.linalg.inv(cov)
    # Calcula a distância de Mahalanobis entre os dois vetores
    dist = np.sqrt(np.dot(np.dot((x1 - x2), cov_inv), (x1 - x2).T))
    return dist

# Define o modelo
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(altura, largura, canais)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(embedding_size, activation=None))

# Define a função de loss
def triplet_loss(y_true, y_pred, alpha=0.2):
    # Separa as embeddings em âncora, positivo e negativo
    anchor, positive, negative = tf.split(y_pred, 3, axis=1)
    # Calcula a distância entre a âncora e o positivo
    pos_dist = mahalanobis_dist(anchor, positive)
    # Calcula a distância entre a âncora e o negativo
    neg_dist = mahalanobis_dist(anchor, negative)
    # Calcula a diferença entre as distâncias e adiciona uma margem
    basic_loss = tf.reduce_mean(pos_dist - neg_dist + alpha)
    # Limita o valor mínimo da loss a 0
    loss = tf.maximum(basic_loss, 0.0)
    return loss

# Compila o modelo
model.compile(optimizer='adam', loss=triplet_loss)

# Treina o modelo
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))


##Códigos úteis


Este código irá percorrer cada pasta no diretório do dataset e carregar cada imagem em um array numpy com a forma (112, 112, 3). Em seguida, armazena todas as matrizes de pixel em um array numpy com a forma (número de pastas, 112, 112, 3), e finalmente, salva o array numpy em um arquivo chamado "dataset.npy".





In [ ]:
import os
from PIL import Image
import numpy as np

# Definir o tamanho das imagens
width = 112
height = 112

# Caminho para o diretório do dataset
data_dir = "/caminho/para/o/seu/dataset"

# Lista de todas as pastas no diretório do dataset
folders = os.listdir(data_dir)

# Inicializar o array numpy com zeros
data = np.zeros((len(folders), width, height, 3), dtype=np.uint8)

# Loop através de cada pasta e carregar cada imagem em um array numpy
for i, folder in enumerate(folders):
    # Lista de todas as imagens na pasta
    images = os.listdir(os.path.join(data_dir, folder))
    # Loop através de cada imagem na pasta e carregá-la em um array numpy
    for j, image_name in enumerate(images):
        # Caminho para a imagem
        image_path = os.path.join(data_dir, folder, image_name)
        # Carregar a imagem usando a biblioteca Pillow
        image = Image.open(image_path)
        # Redimensionar a imagem para o tamanho desejado
        image = image.resize((width, height))
        # Converter a imagem em um array numpy
        image = np.array(image)
        # Armazenar o array numpy no array de dados
        data[i, :, :, j] = image

# Salvar o array numpy em um arquivo
np.save("dataset.npy", data)


Abaixo está um exemplo de código em Python utilizando a biblioteca GitPython para fazer o push automático dos arquivos:



In [ ]:
import git

# Diretório com os arquivos a serem enviados
path_to_repo = '/path/to/repo'

# Inicializa o repositório Git
repo = git.Repo(path_to_repo)

# Adiciona os arquivos ao staging
repo.git.add('--all')

# Faz o commit com uma mensagem de descrição
repo.git.commit('-m', 'Adiciona arquivos de dataset')

# Faz o push dos arquivos para o repositório remoto
origin = repo.remote(name='origin')
origin.push()


Para começar a usar o Git LFS, você precisa instalar o Git LFS em sua máquina local. Para isso, siga as instruções na documentação oficial: https://git-lfs.github.com/.

Depois de instalar o Git LFS, você precisará configurá-lo em seu repositório. Você pode fazer isso usando o seguinte comando:

In [ ]:
git lfs install
git lfs track "*.extensão_do_arquivo"
git lfs track "*.jpg"
git lfs track "*.png"
git lfs track "*.mp4"


Existem algumas opções para disponibilizar seu dataset de forma pública no Google Colab:

Armazenar o dataset em um repositório público no GitHub: você pode armazenar seu dataset em um repositório público no GitHub e, em seguida, clonar o repositório no Colab usando o comando !git clone <link do repositório>. Assim, qualquer pessoa com acesso ao link público do Colab pode clonar o repositório e usar o dataset para treinar modelos.